# 石墨烯单层 F吸附
单层石墨烯，上下以F原子吸附，使得Carbon的sp2杂化成为sp3杂化。

## 限制
简单起见增加下列两个限制。
1. 首先考虑F最大浓度覆盖的情况，也就是只要位点能够形sp3杂化，就放上F原子。
2. 同时，增加相邻石墨烯C原子间不能够同一方向放F原子的限制。

在以上两个限制的前提下，生成面积为原胞石墨烯n倍的各种可能情况的结构。这里的n使得总共需要用第一性原理计算的构型数量不大于500.

## 参数
石墨烯的C-C键长为1.41797A
设定F原子初始到graphene平面的z方向距离为2A

## 算法
- Step1：给出要考察的石墨烯单层超胞supercell，也就是给定特定的n，形状选用无非对角元的拓展矩阵
- Step2：获取supercell上C原子的X,Y坐标，分别在z方向上+或者-一定距离设定空位，标定可占据位点。位点分为up和down两组
- Step3：在up组，用F元素替换空位占据点，得到所有up单侧F取代的构型。
- Step3-1：占据后使用构型过滤，得到所有up侧F不相邻的构型。
- Step4：对每一个up单侧的构型，获取down侧的可能取代位点，用空位标记。
- Step4-1：在down组新的标记中，用F原子替换，得到所有可能构型。
- Step4-2：占据后使用构型过滤，得到所有down侧F不相邻的构型。

以上算法中，可能出现这样的问题，既石墨烯翻转后，上下侧没有区别，会出现相同的构型。
为了避免这种情况的出现，在up侧替换的时候，替换的数量限制在<=1/2的位点数量，这样就能够保证构型不会重复。

In [1]:
from sagar.crystal.derive import ConfigurationGenerator
from sagar.io.vasp import read_vasp, write_vasp
from sagar.crystal.structure import MutableCell, car_to_frac
from sagar.crystal.filter import MinDistanceFilter

import os
import shutil
import numpy

In [2]:
cell = read_vasp("./graphene.vasp")
print("The primitive graphene cell is: \n" + str(cell))

The primitive graphene cell is: 
Lattice:
   a:  2.456000  0.000000  0.000000
   b: -1.228000  2.126958  0.000000
   c:  0.000000  0.000000 14.000000
Sites:
 0.000000  0.000000  0.500000 C
 0.666667  0.333333  0.500000 C


### Step1 扩胞

使用下列矩阵扩为更大的超胞，用于后续的替换

In [3]:
mat = numpy.array([2,0,0,
                   0,2,0,
                   0,0,1]).reshape((3,3))

n_carbon = 8
supercell = cell.extend(mat)

### Step2 
在该石墨烯层的上方放置全满的占位标识符Vacc

In [4]:
# 将结构转化为一个可变的类型MutableCell
mscell = MutableCell.from_cell(supercell)
print(mscell)

Lattice:
   a:  4.912000  0.000000  0.000000
   b: -2.456000  4.253917  0.000000
   c:  0.000000  0.000000 14.000000
Sites:
[0]: 0.000000  0.000000  0.500000 C
[1]: 0.000000  0.500000  0.500000 C
[2]: 0.500000  0.000000  0.500000 C
[3]: 0.500000  0.500000  0.500000 C
[4]: 0.333333  0.166667  0.500000 C
[5]: 0.333333  0.666667  0.500000 C
[6]: 0.833333  0.166667  0.500000 C
[7]: 0.833333  0.666667  0.500000 C


In [5]:
# 向z轴正方向平移2A单位，平移矢量为trnas
lattice = mscell._lattice
trans = numpy.array([0.0, 0.0, 2])

for i in range(n_carbon):
    s = mscell.get_car_site(i)
    car_pos = numpy.array(s[0]) + trans
    pos = car_to_frac(lattice, car_pos)
    mscell.add_site([tuple(pos), 'Vacc'])

print(mscell)
up_cell = mscell.to_cell()

Lattice:
   a:  4.912000  0.000000  0.000000
   b: -2.456000  4.253917  0.000000
   c:  0.000000  0.000000 14.000000
Sites:
[0]: 0.000000  0.000000  0.500000 C
[1]: 0.000000  0.500000  0.500000 C
[2]: 0.500000  0.000000  0.500000 C
[3]: 0.500000  0.500000  0.500000 C
[4]: 0.333333  0.166667  0.500000 C
[5]: 0.333333  0.666667  0.500000 C
[6]: 0.833333  0.166667  0.500000 C
[7]: 0.833333  0.666667  0.500000 C
[8]: 0.000000  0.000000  0.642857 Vacc
[9]: 0.000000  0.500000  0.642857 Vacc
[10]: 0.500000  0.000000  0.642857 Vacc
[11]: 0.500000  0.500000  0.642857 Vacc
[12]: 0.333333  0.166667  0.642857 Vacc
[13]: 0.333333  0.666667  0.642857 Vacc
[14]: 0.833333  0.166667  0.642857 Vacc
[15]: 0.833333  0.666667  0.642857 Vacc


### Step 3 up侧加入F元素
#### Step3-1 以上构型中包含F元素相邻的情况，过滤该种结构

在Step2中产生的up侧加入F原子，最大取代数量为总可占据数的一半。保证翻面后没有重复构型。
同时限制替换的F原子不相邻，使用MinDistanceFilter过滤器

In [6]:
# 使用MinDistanceFilter进行结构筛选
# 其中最近C-C键的键长为1.41797A，次近C-C键长为2.456A
# 设定标度为1.5,表示当结构中全部F-F原子的距离不小于1.5时该结构可以接受，且次近可接受
md_filter = MinDistanceFilter('F', 1.5)

In [7]:
symprec = 1e-3
up_cg = ConfigurationGenerator(up_cell, symprec=symprec)
up_sites = [(6,)]*n_carbon + [(9, 0)]*n_carbon

up_confs = []
for i in range(1, n_carbon//2+1):
    confs = up_cg.cons_specific_cell(sites=up_sites, e_num=(i, n_carbon-i), symprec=symprec)
    for c, _ in confs:
        if md_filter.is_accepted(c):
            up_confs.append(c)
        
# 取代单面，有这样多个结构
print("The number of single side occupy: {:}".format(len(up_confs)))

The number of single side occupy: 5


In [8]:
# # 创建保存结构的文件夹，若存在，则删除
# path = './graphene-F-doped2x2-onside'
# comment = 'F_doped_Graphene-onside'
# if not os.path.exists(path):
#     os.mkdir(path)
# else:
#     shutil.rmtree(path)
#     os.mkdir(path)

In [9]:
# # 写入
# idx = 0
# for c in up_confs:
#     filename = '{:s}_id{:d}'.format(comment, idx)
#     idx += 1
#     file = os.path.join(path, filename)
#     write_vasp(c, file)

### Step4：对每一个up单侧的构型，获取down侧的可能取代位点，用空位标记

In [10]:
# 所有Vacc位置沿z轴负方向平移4A
trans = numpy.array([0.0, 0.0, -4])

# 变为MutableCell对象
up_confs = [MutableCell.from_cell(c) for c in up_confs]
up_confs_vaccdown = []

# TODO: 在src中增加对一群原子的变换
# 对每一个构型做down位点标识
for c in up_confs:
    lattice = c._lattice
    n = len(c._sites)
    
    # 对每一个site迭代找出Vacc的位置
    for idx in range(n):
        s = c.get_car_site(idx)
        if s[1] == 'Vacc':
            car_pos = numpy.array(s[0]) + trans
            pos = car_to_frac(lattice, car_pos)
            c.set_site(idx, [tuple(pos), 'Vacc'])
        
    up_confs_vaccdown.append(c)

In [11]:
# 使用不可变的Cell类做替换
up_confs_vaccdown = [c.to_cell() for c in up_confs_vaccdown]

#### Step4-1：在down组新的标记中，用F原子替换，得到所有可能构型
#### Step4-2：占据后使用构型过滤，得到所有down侧F不相邻的构型
该步骤同Step3相同，同样用到Step3中定义的MinDistanceFilter过滤器

In [12]:
from sagar.crystal.structure import symbol2number as s2n
def _get_sites(l_atoms, ele, l_sub):
    ele_n = s2n(ele)
    l_sub_n = [s2n(sub_n) for sub_n in l_sub]
    sites = []
    for a in l_atoms:
        if a == ele_n:
            sites.append(tuple([a] + l_sub_n))
        else:
            sites.append(tuple([a]))
    return sites

In [13]:
symprec = 1e-3
updown_confs = []
element = 'Vacc'
substitute = 'F'
for vacc_conf in up_confs_vaccdown:
    cg = ConfigurationGenerator(vacc_conf, symprec=symprec)
    # 获取当前需要取代的无需点的位置信息
    sites = _get_sites(vacc_conf.atoms.tolist(), element, substitute)
    
    confs = cg.cons_specific_cell(sites=sites, e_num=None, symprec=symprec)
    for c, _ in confs:
        if md_filter.is_accepted(c):
            updown_confs.append(c)

In [14]:
# 取代双面，有这样多个结构
print("The number of double side occupy: {:}".format(len(updown_confs)))

The number of double side occupy: 44


### Step Optinal: 将文件写入文件夹，写为vasp读取的格式

In [15]:
# 创建保存结构的文件夹，若存在，则删除
path = './graphene-F-doped2x2'
comment = 'F_doped_Graphene'
if not os.path.exists(path):
    os.mkdir(path)
else:
    shutil.rmtree(path)
    os.mkdir(path)

In [16]:
# 写入
idx = 0
for c in updown_confs:
    filename = '{:s}_id{:d}'.format(comment, idx)
    idx += 1
    file = os.path.join(path, filename)
    write_vasp(c, file)